In [1]:
import pandas as pd
import pyodbc

In [2]:
# Connect to DataBank
connection_string = (
    "DSN=DataBank;"
    "Trusted_Connection=yes;"
)

con = pyodbc.connect(connection_string)

In [3]:
spend = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_spend;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_2848\684459653.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spend = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_spend;", con)


In [4]:
spend['paymentdate_financial_year'].isna().sum()

(spend['paymentdate_financial_year'] == 'Unknown').sum()

print(spend[spend['paymentdate_financial_year'] == 'Unknown']['amount'].sum())

print(spend['amount'].sum())

1241783511.5/74191853092.41724 # ~2% of spend on the SSD has Unknown date so can't be accounted for 

1241783511.5
74191853092.41724


0.016737464556292585

In [5]:
spend1 = spend.loc[spend['paymentdate_financial_year'] != 'Unknown']

spend2 = spend1.loc[spend1['paymentdate_financial_year'].str[:4].astype(float) > 2018] # spend from fy 2018/19 onwards


In [6]:
df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_application;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_2848\108949258.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_application;", con)


In [7]:
df_application[['application_reference', 'application_source']].duplicated().sum()

19

In [8]:
df_research = df_application.loc[df_application['application_category'] == 'Research Grant'] # Research Grant only

In [9]:
data_w_spend = pd.merge(spend2, df_research.drop_duplicates(subset=['application_reference', 'application_source'], keep=False), 
                      on=['application_reference', 'application_source'], how='outer', indicator=True)

In [10]:
data_w_spend[data_w_spend['_merge'] == 'left_only']['amount'].sum() # £4.6M spend not linked to application

17704138704.775696

In [11]:
data_both = data_w_spend.loc[data_w_spend['_merge'] == 'both']

In [12]:
data_select = data_both[['award_reference_x', 'application_reference', 'paymentdate_financial_year', 'ExpenditureType', 'amount', 'application_source', 
              'funder_x', 'application_category', 'decision_date']]

In [13]:
df_application['decision_date'].isna().sum()

102955

In [14]:
len(df_application)

582874

In [15]:
102955/582874

0.1766333718779702

In [16]:
print(data_select[['application_reference', 'application_source', 'decision_date']].drop_duplicates()['decision_date'].isna().sum())

print(len(data_select[['application_reference', 'application_source', 'decision_date']].drop_duplicates()))

91/26814*100

91
26814


0.33937495338256135

In [17]:
councils = ['NERC', 'BBSRC', 'STFC', 'MRC', 'ESRC', 'AHRC', 'EPSRC']

data_councils = data_select.loc[data_select['funder_x'].isin(councils)]


In [18]:
print(data_councils[['application_reference', 'application_source', 'decision_date']].drop_duplicates()['decision_date'].isna().sum())

print(len(data_councils[['application_reference', 'application_source', 'decision_date']].drop_duplicates()))

90/22021*100

90
22021


0.40870078561373235

In [41]:
print(data_councils[data_councils['decision_date'].isna()]['amount'].sum())

print(data_councils['amount'].sum())

data_councils.dropna(subset='decision_date', inplace=True)

30602739.89/9072451116.976799*100 # <1% of amount decision date is nan so can neglect

30602739.89
9072451116.976799


C:\Users\MHil02\AppData\Local\Temp\ipykernel_2848\3658491330.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_councils.dropna(subset='decision_date', inplace=True)


0.33731501548390497

In [42]:
data_councils['app_ref_source'] = data_councils['application_reference'] + '_' + data_councils['application_source']

data_1 = data_councils.groupby('paymentdate_financial_year', as_index=False).agg({'app_ref_source':'nunique', 'amount':'sum'})

C:\Users\MHil02\AppData\Local\Temp\ipykernel_2848\2050925763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_councils['app_ref_source'] = data_councils['application_reference'] + '_' + data_councils['application_source']


In [43]:
data_councils['decision_fy'] = data_councils['decision_date'].dt.year

data_councils.loc[data_councils['decision_date'].dt.month <= 3, 'decision_fy'] = data_councils['decision_date'].dt.year -1

data_2 = data_councils.groupby('decision_fy', as_index=False).agg({'app_ref_source':'nunique', 'amount':'sum'})

C:\Users\MHil02\AppData\Local\Temp\ipykernel_2848\842042223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_councils['decision_fy'] = data_councils['decision_date'].dt.year


In [44]:
data_2['Financial Year'] = data_2['decision_fy'].astype(int).astype(str) + '/' + (data_2['decision_fy'].astype(int)+1).astype(str).str[2:]


In [45]:
data_2.drop(columns='decision_fy', inplace=True)
data_2.rename(columns={'app_ref_source':'Decisions', 'amount':'Spend (on decisions)'}, inplace=True)

In [46]:
data_2

data_1.rename(columns={'paymentdate_financial_year':'Financial Year', 'app_ref_source':'Active Awards', 'amount':'Spend'}, inplace=True)

In [47]:
pd.merge(data_1, data_2, on='Financial Year').to_clipboard(index=False)

In [48]:
data_councils.groupby(['app_ref_source'], as_index=False).agg({'amount':'sum'}).sort_values(by='amount', ascending=False).head(20)['amount'].sum()

578308564.3219999

In [49]:
data_1

,Financial Year,Active Awards,Spend
0,2019/20,10148,1.583662e+09
1,2020/21,9143,1.581732e+09
2,2021/22,9690,1.422793e+09
3,2022/23,10105,1.590199e+09
4,2023/24,10007,1.616437e+09
5,2024/25,8385,1.247025e+09


In [50]:
data_2

,Decisions,Spend (on decisions),Financial Year
0,1,0.000000e+00,2006/07
1,2,-4.770023e+04,2008/09
2,4,4.910987e+04,2009/10
3,7,8.132512e+05,2010/11
4,26,2.552055e+05,2011/12
5,162,2.605585e+07,2012/13
6,335,3.536517e+07,2013/14
7,776,1.115579e+08,2014/15
8,1688,2.250017e+08,2015/16
9,2331,6.646956e+08,2016/17


In [51]:
data_councils

data_councils.groupby('decision_fy', as_index=False).agg({'app_ref_source':'nunique', 'amount':'sum'})

,decision_fy,app_ref_source,amount
0,2006,1,0.000000e+00
1,2008,2,-4.770023e+04
2,2009,4,4.910987e+04
3,2010,7,8.132512e+05
4,2011,26,2.552055e+05
5,2012,162,2.605585e+07
6,2013,335,3.536517e+07
7,2014,776,1.115579e+08
8,2015,1688,2.250017e+08
9,2016,2331,6.646956e+08


In [52]:
data_councils['decision_fy'].isna().sum()


0

In [54]:
data_councils

data_councils['Financial Year'] = data_councils['decision_fy'].astype(int).astype(str) + '/' + (data_councils['decision_fy'].astype(int)+1).astype(str).str[2:]


C:\Users\MHil02\AppData\Local\Temp\ipykernel_2848\3713263867.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_councils['Financial Year'] = data_councils['decision_fy'].astype(int).astype(str) + '/' + (data_councils['decision_fy'].astype(int)+1).astype(str).str[2:]


In [59]:
data_councils.loc[data_councils['Financial Year'] == data_councils['paymentdate_financial_year']].groupby('Financial Year', as_index=False).agg({'app_ref_source':'nunique', 'amount':'sum'}).to_clipboard(index=False)


In [60]:
data_councils.groupby('Financial Year', as_index=False).agg({'app_ref_source':'nunique', 'amount':'sum'})


,Financial Year,app_ref_source,amount
0,2006/07,1,0.000000e+00
1,2008/09,2,-4.770023e+04
2,2009/10,4,4.910987e+04
3,2010/11,7,8.132512e+05
4,2011/12,26,2.552055e+05
5,2012/13,162,2.605585e+07
6,2013/14,335,3.536517e+07
7,2014/15,776,1.115579e+08
8,2015/16,1688,2.250017e+08
9,2016/17,2331,6.646956e+08


In [61]:
data_councils.loc[data_councils['Financial Year'] != data_councils['paymentdate_financial_year']]

,award_reference_x,application_reference,paymentdate_financial_year,ExpenditureType,amount,application_source,funder_x,application_category,decision_date,app_ref_source,decision_fy,Financial Year
222329,AH/J006513/1,AH/J006513/1,2019/20,Exception - Staff,0.00,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222330,AH/J006513/1,AH/J006513/1,2019/20,Indirect - Indirect Costs,0.29,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222331,AH/J006513/1,AH/J006513/1,2019/20,DA - Other Directly Allocated,0.00,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222332,AH/J006513/1,AH/J006513/1,2019/20,DI - T&S,-843.88,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222333,AH/J006513/1,AH/J006513/1,2019/20,DA - Investigators,0.00,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
...,...,...,...,...,...,...,...,...,...,...,...,...
2047995,ST/Z000300/1,ST/Z000300/1,2024/25,DI - Equipment,8000.00,Siebel,STFC,Research Grant,2024-02-14,ST/Z000300/1_Siebel,2023,2023/24
2047996,ST/Z000424/1,ST/Z000424/1,2024/25,DI - T&S,520.09,Siebel,STFC,Research Grant,2024-03-22,ST/Z000424/1_Siebel,2023,2023/24
2047997,ST/Z000424/1,ST/Z000424/1,2024/25,DI - T&S,520.09,Siebel,STFC,Research Grant,2024-03-22,ST/Z000424/1_Siebel,2023,2023/24
2047998,ST/Z000424/1,ST/Z000424/1,2024/25,DI - T&S,520.09,Siebel,STFC,Research Grant,2024-03-22,ST/Z000424/1_Siebel,2023,2023/24


In [63]:
data_councils[data_councils['application_reference'] == 'AH/J006513/1']

,award_reference_x,application_reference,paymentdate_financial_year,ExpenditureType,amount,application_source,funder_x,application_category,decision_date,app_ref_source,decision_fy,Financial Year
222329,AH/J006513/1,AH/J006513/1,2019/20,Exception - Staff,0.00,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222330,AH/J006513/1,AH/J006513/1,2019/20,Indirect - Indirect Costs,0.29,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222331,AH/J006513/1,AH/J006513/1,2019/20,DA - Other Directly Allocated,0.00,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222332,AH/J006513/1,AH/J006513/1,2019/20,DI - T&S,-843.88,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222333,AH/J006513/1,AH/J006513/1,2019/20,DA - Investigators,0.00,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222334,AH/J006513/1,AH/J006513/1,2019/20,DI - Staff,-0.02,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222335,AH/J006513/1,AH/J006513/1,2019/20,DI - Other Costs,-1676.16,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222336,AH/J006513/1,AH/J006513/1,2019/20,Exception - Other Costs,0.00,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222337,AH/J006513/1,AH/J006513/1,2019/20,DA - Estate Costs,0.00,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
222338,AH/J006513/1,AH/J006513/1,2019/20,DI - Absence,0.00,Siebel,AHRC,Research Grant,2012-05-31,AH/J006513/1_Siebel,2012,2012/13
